In [16]:
import pandas as pd
from tqdm import tqdm
import transformers
import torch
import torch.nn.functional as F
import numpy as np
import random

In [6]:
df_main = pd.read_csv('10013755/multitude.csv')
df_main.head()

,text,label,multi_label,split,language,length,source
0,Der Ausbruch des Coronavirus hat die Entwicklu...,1,text-davinci-003,test,de,174,MULTITuDE_MassiveSumm_spiegel
1,Alex Azar was officially sworn in as the U.S. ...,1,text-davinci-003,train,en,57,MULTITuDE_MassiveSumm_voanews
2,Європейський союз вимагає зупинити розтрату ко...,1,gpt-3.5-turbo,test,uk,105,MULTITuDE_MassiveSumm_interfax
3,"Yesterday, hundreds of Zambian university stud...",1,text-davinci-003,train,en,254,MULTITuDE_MassiveSumm_voanews
4,"In a narrow and highly watched vote, the US Se...",1,gpt-4,train,en,416,MULTITuDE_MassiveSumm_voanews


In [8]:
from sklearn.metrics import auc,roc_curve
def get_roc_metrics(human_preds, ai_preds):
    # human_preds is the ai-generated probabiities of human-text
    # ai_preds is the ai-generated probabiities of AI-text
    if not human_preds or not ai_preds:
            # Handle empty arrays to avoid the IndexError
            return None    # Rest of your code
    fpr, tpr, _ = roc_curve([0] * len(human_preds) + [1] * len(ai_preds), human_preds + ai_preds, pos_label=1)
    roc_auc = auc(fpr, tpr)
    return fpr.tolist(), tpr.tolist(), float(roc_auc)

Languages available are : 
- English (en)
- Spanish (es)  
- Russian (ru) 
- Dutch (nl)  
- Catalan (ca)   
- Czech (cs)  
- German (de)   
- Chinese (zh) 
- Portuguese (pt)   
- Arabic (ar) 
- Ukrainian (uk)  


testing on the test split of every language

In [ ]:

device = "cuda"# example: cuda:0
detector_path_or_id = "TrustSafeAI/RADAR-Vicuna-7B"
detector = transformers.AutoModelForSequenceClassification.from_pretrained(detector_path_or_id)
tokenizer = transformers.AutoTokenizer.from_pretrained(detector_path_or_id)
detector.eval()
detector.to(device)

In [15]:
languages = ['de', 'en', 'es', 'nl', 'pt', 'ru', 'zh', 'ar', 'uk', 'cs', 'ca']
for i in languages:
    df = df_main[df_main['language'] == i]
    df_test = df[df['split'] == 'test']
    df_test_human = df_test[df_test['label'] == 0]
    df_test_ai = df_test[df_test['label'] == 1]
    print('started evaluating for language:', i)
    output_probs_human = []
    output_probs_ai = []
    with torch.no_grad():
        for i  in tqdm(df_test_human['text']):
            inputs = tokenizer(i, padding=True, truncation=True, max_length=512, return_tensors="pt")
            inputs = {k:v.to(device) for k,v in inputs.items()}
            output_prob = F.log_softmax(detector(**inputs).logits,-1)[:,0].exp().tolist()
            output_probs_human.append(output_prob)

        for i  in tqdm(df_test_ai['text']):
            inputs = tokenizer(i, padding=True, truncation=True, max_length=512, return_tensors="pt")
            inputs = {k:v.to(device) for k,v in inputs.items()}
            output_prob = F.log_softmax(detector(**inputs).logits,-1)[:,0].exp().tolist()
            output_probs_ai.append(output_prob)
    fpr, tpr, roc_auc = get_roc_metrics(output_probs_human, output_probs_ai)
    print('roc_auc:', roc_auc)
    print('fpr:', fpr)
    print('tpr:', tpr)
    
 

started evaluating for language: de


NameError: name 'tqdm' is not defined